In [1]:
import vtk
import glob

In [2]:
path = r"\\wsl$\Ubuntu-18.04\home\arlenlex\LIGGGHTS_SEAICE\lexi_tests\bond_tension\post"

In [3]:
test_file = sorted(glob.glob(path + r'\*.liggghts'))[3]

In [ ]:
def read_liggghts_dump(filename):
    """
    Reads a LIGGGHTS dump file and extracts particle data.
    Returns numpy arrays of particle positions and radii.
    """
    positions = []
    radii = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        data_started = False
        for line in lines:
            if line.startswith('ITEM: ATOMS'):
                data_started = True
                continue
            if data_started:
                parts = line.split()
                # Adjust indices based on your file format
                x, y, z = float(parts[2]), float(parts[3]), float(parts[4])
                radius = float(parts[-1])  # Assuming the radius is the last column
                positions.append([x, y, z])
                radii.append(radius)
    return np.array(positions), np.array(radii)

def visualize_particles(positions, radii):
    """
    Visualizes particles using VTK.
    """
    # Create a vtkPoints object to store particle positions
    vtk_points = vtk.vtkPoints()
    for pos in positions:
        vtk_points.InsertNextPoint(pos)
    
    # Create a vtkPolyData object to store points data
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)
    
    # Create spheres at each point
    glyph_source = vtk.vtkSphereSource()
    glyph_source.SetRadius(1.0)
    glyph = vtk.vtkGlyph3D()
    glyph.SetSourceConnection(glyph_source.GetOutputPort())
    glyph.SetInputData(poly_data)
    glyph.SetScaleModeToScaleByScalar()
    glyph.SetScaleFactor(1.0)
    
    # Map radii to glyphs
    radii_vtk = vtk.vtkFloatArray()
    radii_vtk.SetName("radii")
    for radius in radii:
        radii_vtk.InsertNextValue(radius)
    poly_data.GetPointData().SetScalars(radii_vtk)
    
    glyph_mapper = vtk.vtkPolyDataMapper()
    glyph_mapper.SetInputConnection(glyph.GetOutputPort())
    
    actor = vtk.vtkActor()
    actor.SetMapper(glyph_mapper)
    
    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)
    
    # Add the actor to the scene
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.2, 0.4)  # Background color
    
    # Render and interact
    render_window.Render()
    render_window_interactor.Start()

# Main function to read the dump file and visualize particles
if __name__ == "__main__":
    filename = test_file
    positions, radii = read_liggghts_dump(filename)
    visualize_particles(positions, radii)